## 日本語 Tokenizer

pip install SudachiPy sudachidict_core # or sudachidict_full


In [9]:
# pip install janome
from janome.tokenizer import Tokenizer
from pprint import pprint

# Janomeのインスタンス化
tokenizer = Tokenizer()

text = '今日は、良い天気です。'
tokens = list(tokenizer.tokenize(text)) # generator -> list
pprint([t.surface for t in tokens])
pprint([(t.surface, t.part_of_speech) for t in tokens])

['今日', 'は', '、', '良い', '天気', 'です', '。']
[('今日', '名詞,副詞可能,*,*'),
 ('は', '助詞,係助詞,*,*'),
 ('、', '記号,読点,*,*'),
 ('良い', '形容詞,自立,*,*'),
 ('天気', '名詞,一般,*,*'),
 ('です', '助動詞,*,*,*'),
 ('。', '記号,句点,*,*')]


In [17]:
# pip install SudachiPy sudachidict_core
from sudachipy import Dictionary
from pprint import pprint

# Stachiのインスタンス化
tokenizer = Dictionary().create()

text = '今日は、良い天気です。'
tokens = tokenizer.tokenize(text)
pprint([t.surface() for t in tokens])
pprint([(t.surface(), t.part_of_speech()) for t in tokens])

['今日', 'は', '、', '良い', '天気', 'です', '。']
[('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
